In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ratings = pd.read_table('ratings.dat',sep='::',names=['user_id','movie_id','rating','time'])
users = pd.read_table('users.dat',sep='::',names=['user_id','gender','age','occupation','zip'])
movies = pd.read_table('movies.dat',sep='::',names=['movie_id','title','genre'])


D:\anaconda2\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
D:\anaconda2\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
D:\anaconda2\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [2]:
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year # As there are some NaN years, resulting type will be float (decimals)
movies.title = movies.title.str[:-7]


genres_unique = pd.DataFrame(movies.genre.str.split('|').tolist()).stack().unique()
genres_unique = pd.DataFrame(genres_unique, columns=['genre']) # Format into DataFrame to store later
movies = movies.join(movies.genre.str.get_dummies().astype(int))
movies.drop('genre', inplace=True, axis=1)



movielens_dataset = pd.merge(pd.merge(ratings,users),movies)


In [3]:
movielens_dataset.title.value_counts()[:25]

American Beauty                                   3428
Star Wars: Episode IV - A New Hope                2991
Star Wars: Episode V - The Empire Strikes Back    2990
Star Wars: Episode VI - Return of the Jedi        2883
Jurassic Park                                     2672
Saving Private Ryan                               2653
Terminator 2: Judgment Day                        2649
Matrix, The                                       2590
Back to the Future                                2583
Silence of the Lambs, The                         2578
Men in Black                                      2538
Raiders of the Lost Ark                           2514
Fargo                                             2513
Sixth Sense, The                                  2459
Braveheart                                        2443
Shakespeare in Love                               2369
Princess Bride, The                               2318
Schindler's List                                  2304
L.A. Confi

In [4]:
movie_stats = movielens_dataset.groupby('title').agg({'rating': [np.size, np.mean]})
movie_stats.head()


rating          
                         size      mean
title                                  
$1,000,000 Duck            37  3.027027
'Night Mother              70  3.371429
'Til There Was You         52  2.692308
'burbs, The               303  2.910891
...And Justice for All    199  3.713568

In [5]:
movie_stats.sort_values([('rating', 'mean')], ascending=False).head(10)

rating     
                                     size mean
title                                         
Smashing Time                           2  5.0
Song of Freedom                         1  5.0
One Little Indian                       1  5.0
Ulysses (Ulisse)                        1  5.0
Baby, The                               1  5.0
Follow the Bitch                        1  5.0
Schlafes Bruder (Brother of Sleep)      1  5.0
Gate of Heavenly Peace, The             3  5.0
Bittersweet Motel                       1  5.0
Lured                                   1  5.0

In [6]:
atleast_100 = movie_stats['rating']['size'] >= 100
movie_stats[atleast_100].sort_values([('rating', 'mean')], ascending=False)[:15].plot(kind = 'barh',legend = False)


In [7]:
movie_stats[atleast_100].sort_values([('rating', 'mean')], ascending=False)[:15].plot(kind = 'barh',legend = False)


In [8]:
#Age plot of Users
movielens_dataset.age.plot.hist(bins=30)
plt.title("Distribution of users' ages")
plt.ylabel('count of users')
plt.xlabel('age')

Text(0.5,0,u'age')

In [9]:
movielens_dataset.year.plot.hist(bins = 30)
plt.title("Distribution of Movies' Yearly")
plt.xlabel("Year")

Text(0.5,0,u'Year')

In [10]:
dfe = movielens_dataset[['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary','Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']]
dfe.columns
asd = []
for column in dfe:
    asd.append(dfe[column].sum())
fig, ax = plt.subplots()    
width = 0.75 # the width of the bars 
ind = [u'Action', u'Adventure', u'Animation', u'Childrens', u'Comedy',
       u'Crime', u'Documentary', u'Drama', u'Fantasy', u'Film-Noir', u'Horror',
       u'Musical', u'Mystery', u'Romance', u'Sci-Fi', u'Thriller', u'War',
       u'Western']  # the x locations for the groups
ax.barh(ind, asd, color="blue")
plt.title('Genres Popularity')
plt.xlabel('No of Movies')

Text(0.5,0,u'No of Movies')

In [11]:
#Low Rated Movies
lower_rated = movie_stats['rating']['mean'] < 1.5

In [12]:
movie_stats[lower_rated].head(10).plot(kind = 'barh',figsize = (8,5))